In [1]:
#Representacion de los estados para la resolaucion del problema de los ascensores. 

#Estado Inicial: 
#          * personas indicando la plata actual en la que estan
#ini = ((1, 3, 4), ((0,[]),(0,[])))
ini = ((1, 3, 4), (1,0))
#                   * Ascensores tambien indicando en la planta actual en la que estan. 


fin = ((2, 4, 1)) #Reresentacion des estado final solo de las personas ya que los ascensores no importa doende 
#acaben 


In [2]:
# importamos las cosas que vamos a usar de aima
from search import *

class Ascensores(Problem):
    ''' Clase Ascensores la cual implenta la clase abstracta Problema que nos ayuda a resover el porblema
           con varias funciones de busqueda '''
    def __init__(self,initial, goal):
        '''Metodo que inicializa nustro problema'''
          Problem.__init__(self, initial, goal)
        #self.search = initial
        self.goal = goal
        

    def actions(self, estado):
        '''Apartir de un estado(n) dado nos lista un conjunto de acciones(a) que le podimos aplicar a n para
        alcanzar otro estado n'  ''' 
        
        listaPersonas = list(estado[0])
        listAscensores = list(estado[1])
        
        acciones = list() 
      
        for i in range (len(listaPersonas)):
            persona = listaPersonas[i]
            
            for j in range (len(listAscensores)):
                ascensor = int(listAscensores[j])
                if persona != ascensor: 
                    acciones.append((i,j, "Llamar")) # sube al ascensor j a la planta donde esta la persona i
                if persona == ascensor and int(persona) < 12: # para si la persona esta en la ultima planta solo
                    acciones.append((i,j, "Bajar"))           #pordria bajar 
                if persona == ascensor and int(persona) > 0: # Idem pero al reves 
                    acciones.append((i,j, "Subir"))
                #en el momento que ascensor este en la misma planta que la persona el arbol se ramifica en dos caminos
                #por lo que se generaran estas dos acciones. Quien decida que accion utilizar será la heuristica 
                #puesto que cogere la que menos heuristica tenga ya que estará mas cerca de la solución
                
        return acciones; 

        
    #accion = (plantaPersona,PlantaAscensor, accion en string)
    def result(self, estado, accion):
        '''Dado un estado(n, nodo actual) y una accion(a) nos permite general un nuevo nodo(n') aplicando la accion dada'''
        l = list(estado)
        listaPersonas = list(estado[0]) #la primera(0)posicion de la tupla de estado la convertimos a una lista
        listAscensores = list(estado[1])# Idema para los ascensores con la segunda posicion (1)
        
        persona = accion[0] # Extraemos la posicion de la listaen donde se encutra la persona a modificar su posicion en el edificio
        ascensor = accion[1]# Idem con los ascensores 
        #Se hace simplemente para facilitar la lectura del codigo
        
        actualPersona = listaPersonas[persona] #Planta actual donde se encutra la persona
        actualAscensor = listAscensores[ascensor]#idem con el ascensor
        
        if accion[2] == "Llamar":
            listAscensores[ascensor] = int(listaPersonas[persona]) # Si es llamar a la posicion actual le asignamos la posicion 
            #actual de la persona que ha solicitado el ascensor
        if accion[2] == "Bajar": #Si es bajar restamos en uno la posicion actual de ambos(bajan un piso)
            listaPersonas[persona] -= 1 
            listAscensores[ascensor] -=1
        if accion[2] == "Subir": #Si es Subir sumamos en uno la posicion actual de ambos(sube un piso)
            listaPersonas[persona] += 1
            listAscensores[ascensor] +=1
        
        return (tuple(listaPersonas), tuple(listAscensores))
    
    def goal_test(self, estado): # Devuelve true si se ha alcanzado el estado objetivo 
        return estado[0] == self.goal 




In [3]:
p = Ascensores(ini, fin)
p.initial

((1, 3, 4), (1, 0))

In [4]:
p.actions(p.initial)

[(0, 0, 'Bajar'),
 (0, 0, 'Subir'),
 (0, 1, 'Llamar'),
 (1, 0, 'Llamar'),
 (1, 1, 'Llamar'),
 (2, 0, 'Llamar'),
 (2, 1, 'Llamar')]

In [5]:
p.result(p.initial, (0,0, "Llamar"))

((1, 3, 4), (1, 0))

In [6]:
# Importamos los modulos pertenecinetes a los algoritmos de busqueda 
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [7]:
breadth_first_graph_search(p).solution()

[(0, 0, 'Subir'),
 (1, 0, 'Llamar'),
 (1, 0, 'Subir'),
 (2, 0, 'Bajar'),
 (2, 0, 'Bajar'),
 (2, 0, 'Bajar')]

In [21]:
#Definimos algunas Hehuristicas 

def linear(node): # dado un estado nos devuelve las personas que todavia no estan en su planta objetivo
# es decir, que estan mal colocadas (como en el puzzle de 8)
    result = 0
    goal = fin
    estado = node.state[0]
    
    for i in range (len(estado)):
        if goal[i] != estado[i]:
            result += 1
    return result

# Sería como el de distancia hasta su objetivo quitando los if que comprueban las paradas para cambio de ascensor
def paradasTotales(node):
    result = 0
    bloque1 = (0,1,2,3,4)
    bloque2 = (4,5,6,7,8)
    bloque3 = (8,9,10,11,12)
    
    goal = fin
    estado = node.state[0]
    
    for i in range (len(estado)):
        plantaIni = estado[i]
        plantaFin = goal[i]
        #podemos ignorar la comporacion de goal[i] == estado[i] ya que el resultado es 0
        result += abs(goal[i] - estado[i])
        
        if goal[i] != estado[i]: 
        #Hago esto con conjuntos para determinar las paradas que han de hacer los ascensores Lentos
        #Por ejemplo si el inicio esta en el bloque 1 y el objetivo es el bloque3 entonces ha de hacer dos paradas
            if plantaIni < plantaFin: #Sube 
                if plantaFin in bloque2 and plantaIni in bloque1: 
                    result += 1
                elif plantaFin in bloque3 and plantaIni in bloque1:
                    result +=2
                elif plantaFin in bloque3 and plantaIni in bloque2:
                    result += 1
            else: # Baja
                if plantaIni in bloque3 and plantaFin in bloque2: 
                    result += 1
                elif plantaIni in bloque3 and plantaFin in bloque1:
                    result +=2
                elif plantaIni in bloque2 and plantaFin in bloque1:
                    result += 1
                
    return result

def distanciaParaObjetivo(node):
    result = 0
    goal = fin
    estado = node.state[0]
    
    for i in range (len(estado)):
        if(goal[i] != estado[i]):
            result += abs(goal[i] - estado[i])
    
    return result
        
    
    
    

    
    

In [107]:
#astar_search(p, distanciaParaObjetivo).solution()